### 자동차 가격 예측
* 자동차의 연식, 차종, 연비, 배기량 등에 따라서 가격이 어떻게 형성되는지 확인하고 가격예측모델 만들기

In [1]:
import pandas as pd

train = pd.read_excel('data/carprice_E1SUl6b.xlsx', sheet_name='train')
print(train.shape)
train.head()

(71, 11)


,가격,년식,종류,연비,마력,토크,연료,하이브리드,배기량,중량,변속기
0,1885,2015,준중형,11.8,172,21.0,가솔린,0,1999,1300,자동
1,2190,2015,준중형,12.3,204,27.0,가솔린,0,1591,1300,자동
2,1135,2015,소형,15.0,100,13.6,가솔린,0,1368,1035,수동
3,1645,2014,소형,14.0,140,17.0,가솔린,0,1591,1090,자동
4,1960,2015,대형,9.6,175,46.0,디젤,0,2497,1990,자동


In [2]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71 entries, 0 to 70
Data columns (total 11 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   가격      71 non-null     int64  
 1   년식      71 non-null     int64  
 2   종류      71 non-null     object 
 3   연비      71 non-null     float64
 4   마력      71 non-null     int64  
 5   토크      71 non-null     float64
 6   연료      71 non-null     object 
 7   하이브리드   71 non-null     int64  
 8   배기량     71 non-null     int64  
 9   중량      71 non-null     int64  
 10  변속기     71 non-null     object 
dtypes: float64(2), int64(6), object(3)
memory usage: 6.2+ KB


In [3]:
print(train['가격'].value_counts())

1270     4
1960     3
2430     2
1895     2
1149     2
2250     2
1104     2
1885     2
2150     2
1850     2
1915     2
1645     1
1560     1
2110     1
4650     1
2340     1
2595     1
3361     1
2080     1
2845     1
1430     1
3990     1
1459     1
3091     1
1955     1
11150    1
2190     1
1164     1
2695     1
1542     1
3254     1
1410     1
3373     1
1519     1
3802     1
2160     1
1610     1
1630     1
1135     1
3838     1
4058     1
4190     1
5463     1
1492     1
3195     1
5710     1
1845     1
3277     1
3024     1
3065     1
2495     1
2366     1
3450     1
2745     1
2054     1
2870     1
3585     1
Name: 가격, dtype: int64


In [4]:
print(train['년식'].value_counts())

2015    54
2014     6
2012     4
2011     4
2013     3
Name: 년식, dtype: int64


In [5]:
# 훈련데이터의 독립변수 추출하기 
X_train = train.loc[:, '년식':'변속기']
print(X_train.shape)
X_train.head()

(71, 10)


,년식,종류,연비,마력,토크,연료,하이브리드,배기량,중량,변속기
0,2015,준중형,11.8,172,21.0,가솔린,0,1999,1300,자동
1,2015,준중형,12.3,204,27.0,가솔린,0,1591,1300,자동
2,2015,소형,15.0,100,13.6,가솔린,0,1368,1035,수동
3,2014,소형,14.0,140,17.0,가솔린,0,1591,1090,자동
4,2015,대형,9.6,175,46.0,디젤,0,2497,1990,자동


In [8]:
# 훈련데이터의 종속변수 추출하기
y_train = train['가격']
print(y_train.shape)
y_train.head()

(71,)


0    1885
1    2190
2    1135
3    1645
4    1960
Name: 가격, dtype: int64

In [9]:
# 검증데이터
test = pd.read_excel('data/carprice_E1SUl6b.xlsx', sheet_name='test')
X_test = test.loc[:, '년식':'변속기']
print(X_test.shape)
y_test = test['가격']
print(y_test.shape)

(31, 10)
(31,)


#### 원-핫 인코딩(One-hot Encoding)
* 머신러닝 알고리즘은 문자열 값을 입력 값으로 허락하지 않는다.
* 그렇기 때문에 모든 문자열 값들을 숫자 형으로 인코딩하는 전처리 작업 후에 머신러닝 모델에 학습을 시켜야한다.

##### get_dummies
왜 더미로 가변수화해야하는걸까?
* 왜냐하면, 수치형 데이터로만 변환을 하게 되면 서로 간의 관계성이 생기게 된다.
* 예를 들어, 월요일을 1, 화요일을 2, 수요일을 3이라고 변환할 때, 수치로 나타내면 1+2 = 3이라는 관계성이 존재한다. 
* 그러나 실제 데이터인 월요일, 화요일, 수요일 간에는 그러한 관계성이 없다!
* 따라서, 사실이 아닌 관계성으로 인해 잘못된 학습이 일어날 수 있으므로
* 서로 무관한 수, 즉 더미로 만든 가변수로 변환함으로서 그러한 문제를 막아준다

In [10]:
# 범주형 데이터를 원핫인코딩 방식으로 변환하기
X_train_dummy = pd.get_dummies(X_train) 
X_test_dummy = pd.get_dummies(X_test)
print(X_train_dummy.columns)

Index(['년식', '연비', '마력', '토크', '하이브리드', '배기량', '중량', '종류_대형', '종류_소형',
       '종류_준중형', '종류_중형', '연료_LPG', '연료_가솔린', '연료_디젤', '변속기_수동', '변속기_자동'],
      dtype='object')


In [11]:
X_train_dummy.head()

,년식,연비,마력,토크,하이브리드,배기량,중량,종류_대형,종류_소형,종류_준중형,종류_중형,연료_LPG,연료_가솔린,연료_디젤,변속기_수동,변속기_자동
0,2015,11.8,172,21.0,0,1999,1300,0,0,1,0,0,1,0,0,1
1,2015,12.3,204,27.0,0,1591,1300,0,0,1,0,0,1,0,0,1
2,2015,15.0,100,13.6,0,1368,1035,0,1,0,0,0,1,0,1,0
3,2014,14.0,140,17.0,0,1591,1090,0,1,0,0,0,1,0,0,1
4,2015,9.6,175,46.0,0,2497,1990,1,0,0,0,0,0,1,0,1


In [13]:
# 모델 생성 학습 및 예측, 선형회귀(LinearRegression)
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

model = LinearRegression()

model.fit(X_train_dummy, y_train)

y_predict = model.predict(X_test_dummy)
score = r2_score(y_test, y_predict)
print(score)

0.7739730315244966


In [14]:
X_test_dummy.head(2)

,년식,연비,마력,토크,하이브리드,배기량,중량,종류_대형,종류_소형,종류_준중형,종류_중형,연료_LPG,연료_가솔린,연료_디젤,변속기_수동,변속기_자동
0,2015,6.8,159,23.0,0,2359,1935,1,0,0,0,1,0,0,1,0
1,2012,13.3,108,13.9,0,1396,1035,0,1,0,0,0,1,0,0,1


In [16]:
import numpy as np
X_new = np.array([[2020,6.8,159,23.0,1,2359,1935,1,0,0,0,0,1,0,0,1]])
X_new
print(model.predict(X_new))

[2105.17304152]


In [20]:
check = lambda x : '음의관계' if x < 0  else '양의관계'
check(10)

'양의관계'

In [22]:
for i in range(len(X_train_dummy.columns)):
    print(f'{X_train_dummy.columns[i]} 의 Coefficient 값은 {model.coef_[i]} 이고 가격과 {check(model.coef_[i])} 입니다.')

년식 의 Coefficient 값은 35.10677128290859     이고 가격과 양의관계 입니다.
연비 의 Coefficient 값은 131.19567852386686     이고 가격과 양의관계 입니다.
마력 의 Coefficient 값은 9.8390719959107     이고 가격과 양의관계 입니다.
토크 의 Coefficient 값은 -7.8407577740024585     이고 가격과 음의관계 입니다.
하이브리드 의 Coefficient 값은 325.6806997930438     이고 가격과 양의관계 입니다.
배기량 의 Coefficient 값은 1.7894052265791913     이고 가격과 양의관계 입니다.
중량 의 Coefficient 값은 0.12844595383012347     이고 가격과 양의관계 입니다.
종류_대형 의 Coefficient 값은 -538.075733719554     이고 가격과 음의관계 입니다.
종류_소형 의 Coefficient 값은 463.8614926014822     이고 가격과 양의관계 입니다.
종류_준중형 의 Coefficient 값은 10.944891596661037     이고 가격과 양의관계 입니다.
종류_중형 의 Coefficient 값은 63.26934952141122     이고 가격과 양의관계 입니다.
연료_LPG 의 Coefficient 값은 288.5612126374469     이고 가격과 양의관계 입니다.
연료_가솔린 의 Coefficient 값은 -212.79830412383197     이고 가격과 음의관계 입니다.
연료_디젤 의 Coefficient 값은 -75.76290851361503     이고 가격과 음의관계 입니다.
변속기_수동 의 Coefficient 값은 -161.29147786073793     이고 가격과 음의관계 입니다.
변속기_자동 의 Coefficient 값은 161.29147786073818     이고 가격과 양의관계 입니다.


In [21]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
import numpy as np

fruits=['사과', '바나나', '수박', '체리', '수박', '체리', '메론']

encoder = LabelEncoder()
encoder.fit(fruits)

labels = encoder.transform(fruits)
labels = labels.reshape(-1,1)

# One-Hot Encoding
oneHot_encoder = OneHotEncoder()
oneHot_encoder.fit(labels)
oneHot_labels = oneHot_encoder.transform(labels)
print('oneHot_labels array')
print(oneHot_labels.toarray())
print('-----------------------------')
print('oneHot_labels shape')
print(oneHot_labels.shape)
print('-----------------------------')

test_df = pd.DataFrame({'fruit':fruits})
test_df

oneHot_labels array
[[0. 0. 1. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0.]]
-----------------------------
oneHot_labels shape
(7, 5)
-----------------------------


,fruit
0,사과
1,바나나
2,수박
3,체리
4,수박
5,체리
6,메론


In [18]:
pd.get_dummies(test_df)

,fruit_메론,fruit_바나나,fruit_사과,fruit_수박,fruit_체리
0,0,0,1,0,0
1,0,1,0,0,0
2,0,0,0,1,0
3,0,0,0,0,1
4,0,0,0,1,0
5,0,0,0,0,1
6,1,0,0,0,0
